In [5]:
import Pkg; 

if split(pwd(),"/")[end] == "prototyping"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMolNotebooks
using MorphoMol
using JLD2
using BenchmarkTools
using Random

In [35]:
function solvation_free_energy_in_bounds(x::Vector{Float64}, template_centers::Matrix{Float64}, radii::Vector{Float64}, rs::Float64, prefactors::AbstractVector, overlap_jump::Float64, overlap_slope::Float64, bounds::Float64, delaunay_eps::Float64)
    if any(0.0 >= e || e >= bounds for e in x[4:6:end]) || any(0.0 >= e || e >= bounds for e in x[5:6:end]) || any(0.0 >= e || e >= bounds for e in x[6:6:end])
        return Inf
    end
    n_mol = length(x) ÷ 6
    n_atoms_per_mol = size(template_centers)[2]
    flat_realization = MorphoMol.Utilities.get_flat_realization(x, template_centers)
    MorphoMol.Energies.solvation_free_energy(flat_realization, n_atoms_per_mol, radii, rs, prefactors, overlap_jump, overlap_slope, delaunay_eps)
end

function summed_alpha_shape_persistence_in_bounds(x::Vector{Float64}, template_centers::Matrix{Float64}, persistence_weights::Vector{Float64}, bounds::Float64)
    if any(0.0 >= e || e >= bounds for e in x[4:6:end]) || any(0.0 >= e || e >= bounds for e in x[5:6:end]) || any(0.0 >= e || e >= bounds for e in x[6:6:end])
        return Inf, Dict{String, Any}()
    end
    points = MorphoMol.Utilities.get_point_vector_realization(x, template_centers)
    pdgm = MorphoMol.Energies.get_alpha_shape_persistence_diagram(points)
    p0 = MorphoMol.Energies.get_total_persistence(pdgm[1], persistence_weights[1])
    p1 = MorphoMol.Energies.get_total_persistence(pdgm[2], persistence_weights[2])
    p2 = MorphoMol.Energies.get_total_persistence(pdgm[3], persistence_weights[3])
    p0 + p1 + p2
end

function summed_interface_persistence_in_bounds(x::Vector{Float64}, template_centers::Matrix{Float64}, radii::Vector{Float64}, persistence_weights::Vector{Float64}, bounds::Float64)
    if any(0.0 >= e || e >= bounds for e in x[4:6:end]) || any(0.0 >= e || e >= bounds for e in x[5:6:end]) || any(0.0 >= e || e >= bounds for e in x[6:6:end])
        return Inf, Dict{String, Any}()
    end
    points = MorphoMol.Utilities.get_point_vector_realization(x, template_centers)
    n_atoms_per_mol = size(template_centers)[2]
    idgm = MorphoMol.Energies.get_interface_persistence_diagram(points, n_atoms_per_mol)
    p0 = MorphoMol.Energies.get_death_by_birth_persistence(idgm[1], persistence_weights[1])
    p1 = MorphoMol.Energies.get_death_by_birth_persistence(idgm[2], persistence_weights[2])
    return p0 + p1
end

summed_interface_persistence_in_bounds (generic function with 1 method)

In [36]:
template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_centers"]
template_radii = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY["6r7m"]["template_radii"]
radii = [template_radii; template_radii]
fsol(x) = solvation_free_energy_in_bounds(x, template_centers, radii, 0.3665, [1.0, 1.0, 1.0, 1.0], 0.0, 0.5, 150.0, 100.0)
asp(x) = summed_alpha_shape_persistence_in_bounds(x, template_centers, [1.0, 1.0, 1.0], 150.0)
ip(x) = summed_interface_persistence_in_bounds(x, template_centers, radii, [1.0, 1.0], 150.0)

ip (generic function with 1 method)

In [38]:
@benchmark fsol($rand(12) * 100.0)

BenchmarkTools.Trial: 20 samples with 1 evaluation.
 Range (min … max):  242.974 ms … 258.286 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     250.458 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   250.747 ms ±   5.183 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁  ▁▁   █    ▁ ▁    ▁    ▁ ▁    ▁        ▁  ▁ █   ▁ ▁ ▁     █  
  █▁▁██▁▁▁█▁▁▁▁█▁█▁▁▁▁█▁▁▁▁█▁█▁▁▁▁█▁▁▁▁▁▁▁▁█▁▁█▁█▁▁▁█▁█▁█▁▁▁▁▁█ ▁
  243 ms           Histogram: frequency by time          258 ms <

 Memory estimate: 915.35 KiB, allocs estimate: 21846.

In [39]:
@benchmark asp($rand(12) * 100.0)

BenchmarkTools.Trial: 25 samples with 1 evaluation.
 Range (min … max):  200.722 ms … 210.358 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     203.547 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   203.851 ms ±   2.469 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █    ▃        ▃ ▃  ▃                  █                        
  █▇▇▁▁█▁▇▁▁▁▁▁▁█▁█▇▁█▇▁▁▁▁▁▇▁▇▁▇▁▁▁▇▇▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  201 ms           Histogram: frequency by time          210 ms <

 Memory estimate: 1.24 MiB, allocs estimate: 12230.

In [40]:
@benchmark ip($rand(12) * 100.0)

BenchmarkTools.Trial: 7 samples with 1 evaluation.
 Range (min … max):  494.614 ms …    2.679 s  ┊ GC (min … max): 0.00% … 0.30%
 Time  (median):     602.463 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   929.387 ms ± 783.570 ms  ┊ GC (mean ± σ):  0.12% ± 0.11%

  █ ▁▁    ▁ ▁                                                 ▁  
  █▁██▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  495 ms           Histogram: frequency by time          2.68 s <

 Memory estimate: 8.21 MiB, allocs estimate: 144131.